In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pytz
from openpyxl.utils import get_column_letter

url = "https://www.ev.or.kr/nportal/buySupprt/initSubsidyPaymentCheckAction.do"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

headers = {"User-Agent": user_agent}

# Send the GET request
response = requests.get(
    url,
    headers=headers,
)
soup = BeautifulSoup(
    response.text,
    "html.parser",
)

# Find the table with class 'table01 fz15'
table = soup.find("table", class_="table01 fz15")

tbody = table.find("tbody")
rows = []

# Extract table rows
for row in tbody.find_all("tr"):
    cells = row.find_all("td")
    selected_cells = [cells[i] for i in range(len(cells)) if i in [0, 1, 2, 5, 6, 7, 8]]
    if not selected_cells:
        continue  # this skips rows without <td> tags
    rows.append([cell.text for cell in selected_cells])


def flatten_and_convert(lst):
    new_list = []
    for item in lst:
        # Check if item contains parentheses indicating it has numbers to extract
        if "(" in item:
            # Extract numbers from the string
            numbers = item.replace("(", " ").replace(")", " ").split()
            for num in numbers:
                # Try to convert each part to an integer, if possible
                try:
                    converted_num = int(num)
                    new_list.append(converted_num)
                except ValueError:
                    # If conversion fails, append the string as is (in case of non-numeric parts)
                    new_list.append(num)
        else:
            # For items without parentheses, add them directly to the new list
            new_list.append(item)
    return new_list


rows = [flatten_and_convert(item) for item in rows]


columns = [
    "시도",
    "지역구분",
    "차종구분",
    "민간공고-전체",
    "민간공고-우선순위",
    "민간공고-법인,기관",
    "민간공고-택시",
    "민간공고-일반",
    "접수대수-전체",
    "접수대수-우선순위",
    "접수대수-법인,기관",
    "접수대수-택시",
    "접수대수-일반",
    "민간공고-전체",
    "출고대수-우선순위",
    "출고대수-법인,기관",
    "출고대수-택시",
    "출고대수-일반",
    "출고잔여대수-전체",
    "출고잔여대수-우선순위",
    "출고잔여대수-법인,기관",
    "출고잔여대수-택시",
    "출고잔여대수-일반",
]


In [2]:
df = pd.DataFrame(data=rows, columns=columns)
# df.to_csv("ev.csv")


In [3]:
# Korean timezone
korean_tz = pytz.timezone('Asia/Seoul')

# Current time in Korean timezone
now_korean = datetime.now(korean_tz)
date = now_korean.strftime('%Y/%m/%d')
time = now_korean.strftime('%H:%M')

# Specify the Excel file path
file_path = 'ev2.xlsx'

# Create an ExcelWriter object with the specified engine
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # If you want to write to an existing sheet or the default one, use `writer.book.active`
    ws = writer.book.create_sheet(title='Sheet1')
    
    # Write metadata
    ws['A1'] = 'Date'
    ws['B1'] = date
    ws['A2'] = 'Time'
    ws['B2'] = time

    # Write DataFrame to the Excel starting from the fourth row
    df.to_excel(writer, sheet_name='Sheet1', startrow=3, index=False)
    
    # Save the workbook temporarily to access the written data
    writer.save()

    # Access the 'All' worksheet again to adjust column widths
    ws = writer.book["All"]

    for column_cells in ws.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        ws.column_dimensions[get_column_letter(column_cells[0].column)].width = length

    # Save the workbook again to finalize changes
    writer.save()

print(f"File '{file_path}' has been saved with Korean time metadata.")

AttributeError: 'OpenpyxlWriter' object has no attribute 'save'

In [ ]:
df

,시도,지역구분,차종구분,민간공고-전체,민간공고-우선순위,"민간공고-법인,기관",민간공고-택시,민간공고-일반,접수대수-전체,접수대수-우선순위,...,민간공고-전체,출고대수-우선순위,"출고대수-법인,기관",출고대수-택시,출고대수-일반,출고잔여대수-전체,출고잔여대수-우선순위,"출고잔여대수-법인,기관",출고잔여대수-택시,출고잔여대수-일반
0,서울,서울특별시,전기승용,7380,738,0,2380,4262,1388,72,...,1142,67,43,91,941,6238,671,0,2289,3321
1,부산,부산광역시,전기승용,2940,300,0,1300,1340,698,36,...,579,35,21,79,444,2361,265,0,1221,896
2,대구,대구광역시,전기승용,3679,368,0,736,2575,682,22,...,578,21,22,72,463,3101,347,0,664,2112
3,인천,인천광역시,전기승용,9467,947,0,947,7573,772,41,...,643,40,15,23,565,8824,907,0,924,7008
4,광주,광주광역시,전기승용,1775,178,0,178,1419,267,11,...,203,10,5,19,169,1572,168,0,159,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,경남,함양군,전기승용,55,6,0,6,43,8,0,...,6,0,2,0,4,49,6,0,6,39
157,경남,거창군,전기승용,104,11,0,11,82,99,11,...,87,8,6,0,73,17,3,0,11,9
158,경남,합천군,전기승용,93,15,0,15,63,27,3,...,24,3,1,0,20,69,12,0,15,43
159,제주,제주특별자치도,전기승용,4000,400,100,400,3100,389,161,...,313,131,10,30,142,3687,269,90,370,2958
